In [1]:
import numpy as np
import cv2
import os
from PIL import Image
import dlib

In [2]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('E:\\NYU_Classes\\DSGA_1016 CCM\\project\\dlib-models\\shape_predictor_5_face_landmarks.dat')

In [3]:
paths = ["./Speech/"]

In [4]:
path = 'C:/Users/Kannan Venkat/Anaconda3/envs/GPU/Lib/site-packages/cv2/data/'
face_cascade = cv2.CascadeClassifier(path + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(path + 'haarcascade_eye.xml')

In [5]:
def shape_to_normal(shape):
    shape_normal = []
    for i in range(0, 5):
        shape_normal.append((i, (shape.part(i).x, shape.part(i).y)))
    return shape_normal
def get_eyes_nose_dlib(shape):
    nose = shape[4][1]
    left_eye_x = int(shape[3][1][0] + shape[2][1][0]) // 2
    left_eye_y = int(shape[3][1][1] + shape[2][1][1]) // 2
    right_eyes_x = int(shape[1][1][0] + shape[0][1][0]) // 2
    right_eyes_y = int(shape[1][1][1] + shape[0][1][1]) // 2
    return nose, (left_eye_x, left_eye_y), (right_eyes_x, right_eyes_y)

In [6]:
def distance(a, b):
    return np.sqrt((a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2)

In [7]:
def cosine_formula(length_line1, length_line2, length_line3):
    cos_a = -(length_line3 ** 2 - length_line2 ** 2 - length_line1 ** 2) / (2 * length_line2 * length_line1)
    return cos_a

In [8]:
def rotate_point(origin, point, angle):
    ox, oy = origin
    px, py = point

    qx = ox + np.cos(angle) * (px - ox) - np.sin(angle) * (py - oy)
    qy = oy + np.sin(angle) * (px - ox) + np.cos(angle) * (py - oy)
    return qx, qy


def is_between(point1, point2, point3, extra_point):
    c1 = (point2[0] - point1[0]) * (extra_point[1] - point1[1]) - (point2[1] - point1[1]) * (extra_point[0] - point1[0])
    c2 = (point3[0] - point2[0]) * (extra_point[1] - point2[1]) - (point3[1] - point2[1]) * (extra_point[0] - point2[0])
    c3 = (point1[0] - point3[0]) * (extra_point[1] - point3[1]) - (point1[1] - point3[1]) * (extra_point[0] - point3[0])
    if (c1 < 0 and c2 < 0 and c3 < 0) or (c1 > 0 and c2 > 0 and c3 > 0):
        return True
    else:
        return False

In [9]:
prototxt_path = os.path.join('E:/NYU_Classes/DSGA_1016 CCM/project/deep-learning-face-detection/deep-learning-face-detection/deploy.prototxt')
caffemodel_path = os.path.join('E:/NYU_Classes/DSGA_1016 CCM/project/deep-learning-face-detection/deep-learning-face-detection/res10_300x300_ssd_iter_140000.caffemodel')

model = cv2.dnn.readNetFromCaffe('E:\\NYU_Classes\\DSGA_1016 CCM\\project\\deep-learning-face-detection\\deep-learning-face-detection\\deploy.prototxt.txt', caffemodel_path)

In [10]:
def horizantal_align(image):
    #image = cv2.resize(image, (227, 227))
    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    rects = detector(gray, 0)
    if len(rects) > 0:
        for rect in rects:
            x = rect.left()
            y = rect.top()
            w = rect.right()
            h = rect.bottom()
            shape = predictor(gray, rect)
                
        shape = shape_to_normal(shape)
        nose, left_eye, right_eye = get_eyes_nose_dlib(shape)

        center_of_forehead = ((left_eye[0] + right_eye[0]) // 2, (left_eye[1] + right_eye[1]) // 2)
        center_pred = (int((x + w) / 2), int((y + y) / 2))

        length_line1 = distance(center_of_forehead, nose)
        length_line2 = distance(center_pred, nose)
        length_line3 = distance(center_pred, center_of_forehead)

        cos_a = cosine_formula(length_line1, length_line2, length_line3)
        angle = np.arccos(cos_a)

        rotated_point = rotate_point(nose, center_of_forehead, angle)
        rotated_point = (int(rotated_point[0]), int(rotated_point[1]))

        if is_between(nose, center_of_forehead, center_pred, rotated_point):
            angle = np.degrees(-angle)
        else:
            angle = np.degrees(angle)

        image = Image.fromarray(image)
        rot_image = np.array(image.rotate(angle))
        flag = 1

        return rot_image,flag
    else:
        flag = 0
        return image, flag


In [12]:
def extract_face(img,filename,sec=0):
    (h, w) = img.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(img, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))

    model.setInput(blob)
    detections = model.forward()
    for i in range(0, detections.shape[2]):
        box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
        (startX, startY, endX, endY) = box.astype("int")

        confidence = detections[0, 0, i, 2]

        # If confidence > 0.5, save it as a separate file
        if (confidence > 0.5):
            frame = img[startY:endY, startX:endX]
            #frame = cv2.resize(frame, (227, 227))
            cv2.imwrite("./images2/"+str(filename)+"-"+str(sec)+".jpg", frame)            
    return(frame)

In [13]:
def getFrame(filename,vidcap,sec):
    vidcap.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
    hasFrames,image = vidcap.read()
    
    if hasFrames:
        rot_image,flag = horizantal_align(image)
        if flag==1:
            extract_face(rot_image,filename,sec)
        
    return hasFrames

In [15]:
file_comp = []

In [16]:
for path in paths:
    folders = os.listdir(path)
    for folder in folders:
        files = os.listdir(str(path+"/"+folder))
        for file in files:
            filename = file.split(".")[0]
            if filename not in file_comp:
                mod = int(filename.split("-")[0])
                if mod ==1:
                    print(filename)
                    cap= cv2.VideoCapture(str(path+"/"+folder+"/"+file))
                    sec = 0
                    frameRate = 0.5
                    success = getFrame(filename,cap,sec)
                    while success:
                        sec = sec + frameRate
                        sec = round(sec, 2)
                        success = getFrame(filename,cap,sec)  

01-01-01-01-01-01-01
01-01-01-01-01-02-01
01-01-01-01-02-01-01
01-01-01-01-02-02-01
01-01-02-01-01-01-01
01-01-02-01-01-02-01
01-01-02-01-02-01-01
01-01-02-01-02-02-01
01-01-02-02-01-01-01
01-01-02-02-01-02-01
01-01-02-02-02-01-01
01-01-02-02-02-02-01
01-01-03-01-01-01-01
01-01-03-01-01-02-01
01-01-03-01-02-01-01
01-01-03-01-02-02-01
01-01-03-02-01-01-01
01-01-03-02-01-02-01
01-01-03-02-02-01-01
01-01-03-02-02-02-01
01-01-04-01-01-01-01
01-01-04-01-01-02-01
01-01-04-01-02-01-01
01-01-04-01-02-02-01
01-01-04-02-01-01-01
01-01-04-02-01-02-01
01-01-04-02-02-01-01
01-01-04-02-02-02-01
01-01-05-01-01-01-01
01-01-05-01-01-02-01
01-01-05-01-02-01-01
01-01-05-01-02-02-01
01-01-05-02-01-01-01
01-01-05-02-01-02-01
01-01-05-02-02-01-01
01-01-05-02-02-02-01
01-01-06-01-01-01-01
01-01-06-01-01-02-01
01-01-06-01-02-01-01
01-01-06-01-02-02-01
01-01-06-02-01-01-01
01-01-06-02-01-02-01
01-01-06-02-02-01-01
01-01-06-02-02-02-01
01-01-07-01-01-01-01
01-01-07-01-01-02-01
01-01-07-01-02-01-01
01-01-07-01-0

01-01-05-01-02-02-07
01-01-05-02-01-01-07
01-01-05-02-01-02-07
01-01-05-02-02-01-07
01-01-05-02-02-02-07
01-01-06-01-01-01-07
01-01-06-01-01-02-07
01-01-06-01-02-01-07
01-01-06-01-02-02-07
01-01-06-02-01-01-07
01-01-06-02-01-02-07
01-01-06-02-02-01-07
01-01-06-02-02-02-07
01-01-07-01-01-01-07
01-01-07-01-01-02-07
01-01-07-01-02-01-07
01-01-07-01-02-02-07
01-01-07-02-01-01-07
01-01-07-02-01-02-07
01-01-07-02-02-01-07
01-01-07-02-02-02-07
01-01-08-01-01-01-07
01-01-08-01-01-02-07
01-01-08-01-02-01-07
01-01-08-01-02-02-07
01-01-08-02-01-01-07
01-01-08-02-01-02-07
01-01-08-02-02-01-07
01-01-08-02-02-02-07
01-01-01-01-01-01-09
01-01-01-01-01-02-09
01-01-01-01-02-01-09
01-01-01-01-02-02-09
01-01-02-01-01-01-09
01-01-02-01-01-02-09
01-01-02-01-02-01-09
01-01-02-01-02-02-09
01-01-02-02-01-01-09
01-01-02-02-01-02-09
01-01-02-02-02-01-09
01-01-02-02-02-02-09
01-01-03-01-01-01-09
01-01-03-01-01-02-09
01-01-03-01-02-01-09
01-01-03-01-02-02-09
01-01-03-02-01-01-09
01-01-03-02-01-02-09
01-01-03-02-0

01-01-01-01-02-01-15
01-01-01-01-02-02-15
01-01-02-01-01-01-15
01-01-02-01-01-02-15
01-01-02-01-02-01-15
01-01-02-01-02-02-15
01-01-02-02-01-01-15
01-01-02-02-01-02-15
01-01-02-02-02-01-15
01-01-02-02-02-02-15
01-01-03-01-01-01-15
01-01-03-01-01-02-15
01-01-03-01-02-01-15
01-01-03-01-02-02-15
01-01-03-02-01-01-15
01-01-03-02-01-02-15
01-01-03-02-02-01-15
01-01-03-02-02-02-15
01-01-04-01-01-01-15
01-01-04-01-01-02-15
01-01-04-01-02-01-15
01-01-04-01-02-02-15
01-01-04-02-01-01-15
01-01-04-02-01-02-15
01-01-04-02-02-01-15
01-01-04-02-02-02-15
01-01-05-01-01-01-15
01-01-05-01-01-02-15
01-01-05-01-02-01-15
01-01-05-01-02-02-15
01-01-05-02-01-01-15
01-01-05-02-01-02-15
01-01-05-02-02-01-15
01-01-05-02-02-02-15
01-01-06-01-01-01-15
01-01-06-01-01-02-15
01-01-06-01-02-01-15
01-01-06-01-02-02-15
01-01-06-02-01-01-15
01-01-06-02-01-02-15
01-01-06-02-02-01-15
01-01-06-02-02-02-15
01-01-07-01-01-01-15
01-01-07-01-01-02-15
01-01-07-01-02-01-15
01-01-07-01-02-02-15
01-01-07-02-01-01-15
01-01-07-02-0

01-01-05-02-01-02-22
01-01-05-02-02-01-22
01-01-05-02-02-02-22
01-01-06-01-01-01-22
01-01-06-01-01-02-22
01-01-06-01-02-01-22
01-01-06-01-02-02-22
01-01-06-02-01-01-22
01-01-06-02-01-02-22
01-01-06-02-02-01-22
01-01-06-02-02-02-22
01-01-07-01-01-01-22
01-01-07-01-01-02-22
01-01-07-01-02-01-22
01-01-07-01-02-02-22
01-01-07-02-01-01-22
01-01-07-02-01-02-22
01-01-07-02-02-01-22
01-01-07-02-02-02-22
01-01-08-01-01-01-22
01-01-08-01-01-02-22
01-01-08-01-02-01-22
01-01-08-01-02-02-22
01-01-08-02-01-01-22
01-01-08-02-01-02-22
01-01-08-02-02-01-22
01-01-08-02-02-02-22
01-01-01-01-01-01-23
01-01-01-01-01-02-23
01-01-01-01-02-01-23
01-01-01-01-02-02-23
01-01-02-01-01-01-23
01-01-02-01-01-02-23
01-01-02-01-02-01-23
01-01-02-01-02-02-23
01-01-02-02-01-01-23
01-01-02-02-01-02-23
01-01-02-02-02-01-23
01-01-02-02-02-02-23
01-01-03-01-01-01-23
01-01-03-01-01-02-23
01-01-03-01-02-01-23
01-01-03-01-02-02-23
01-01-03-02-01-01-23
01-01-03-02-01-02-23
01-01-03-02-02-01-23
01-01-03-02-02-02-23
01-01-04-01-0